In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [0]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
data = spark.read.csv('/FileStore/tables/movielens_ratings.csv', inferSchema=True, header=True)

In [0]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [0]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [0]:
training, test = data.randomSplit([0.8, 0.2])

In [0]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [0]:
model = als.fit(training)

In [0]:
predictions = model.transform(test)

In [0]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      1|   1.0|     7| 0.7090732|
|      2|   2.0|     7|  2.652473|
|      0|   1.0|     8| 2.5752082|
|      4|   3.0|    10|  -1.55298|
|      0|   1.0|    11|-0.9866701|
|      9|   1.0|    11| 2.9374635|
|      9|   1.0|    14| 1.4349556|
|      1|   4.0|    15|0.67457235|
|      2|   1.0|    17| 1.5710192|
|      3|   1.0|    17| 0.9150201|
|      6|   1.0|    17| 1.5407002|
|      2|   1.0|    19|-1.5394251|
|      4|   1.0|    19| 1.6326306|
|      9|   1.0|    22| 1.2768414|
|      0|   1.0|    23| 1.7679785|
|      2|   1.0|    25|  0.974948|
|      9|   1.0|    25| 1.1510146|
|      5|   2.0|    26| 2.6059144|
|      7|   5.0|    26|  3.447308|
|      4|   1.0|    29| 1.6234465|
+-------+------+------+----------+
only showing top 20 rows



In [0]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [0]:
rmse = evaluator.evaluate(predictions)

In [0]:
print('RMSE : ',rmse)

RMSE :  1.7213658524226452


In [0]:
single_user = test.filter(test['userId']==11).select(['movieId', 'userId'])

In [0]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|      9|    11|
|     16|    11|
|     18|    11|
|     20|    11|
|     38|    11|
|     50|    11|
|     61|    11|
|     75|    11|
|     76|    11|
|     86|    11|
|     97|    11|
+-------+------+



In [0]:
recommendations = model.transform(single_user)

In [0]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     38|    11|   3.405823|
|      9|    11|  2.9374635|
|     20|    11|  2.1515443|
|     75|    11|  1.2257878|
|     61|    11|  1.0428058|
|     97|    11| 0.53974444|
|     16|    11| 0.42448828|
|     86|    11| 0.13084331|
|     50|    11|-0.48065543|
|      0|    11| -0.9866701|
|     76|    11|   -1.92258|
|     18|    11| -2.0510643|
+-------+------+-----------+

